参考： [[Kaggle]0から本当に機械学習を理解するために学ぶべきこと～一流のデータサイエンティストを例に～ - Qiita](https://qiita.com/zenonnp/items/9cbb2860505a32059d89)

# ライブラリインポート

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# ↑これはJupyter内で図を描写するためのライブラリ

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# データ読み込み

In [ ]:
train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

# データ構成把握

In [ ]:
print(train_df.columns.values)

![img](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F183674%2F7dff3491-0f16-4f59-8d75-48a93eb781ab.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=da8148e79d460bd2540e2931f3330bd1)

In [ ]:
# preview the data
train_df.head()

In [ ]:
# Get data information

train_df.info()

Ageが714

Cabinが204（明らかに少なすぎるので以後は考慮しない）

Name, Sex, Ticket, Embarkedは数値化されていない

In [ ]:
test_df.info()

Age, Fareが欠損

# データの特徴を把握する

In [ ]:
train_df.describe()

* トレーニングデータセットの合計は891で、タイタニック号に搭乗した乗客の実際の数（2,224）の40％です。
  * へー
* Survived は0または1の値を持つカテゴリカル特徴量です。
  * わかる
* トレーニングデータセットの約38％が実際のSurvived率の32％を代表して生き残りました。
  * へー
* ほとんどの乗客（> 75％）は親または子供と一緒に旅行しなかった。
  * どれ？？
* 乗客のほぼ30％が兄弟姉妹や配偶者を乗せていました。
  * どれ？？
* 運賃は、一部の乗客(< 1%)だけ異常に高く、512ドル払っていました。
  * わかる：Fare
* 65歳から80歳までの高齢者はほとんどいませんでした（< 1％）。
  * 1%はわからんがまあわかる

In [ ]:
train_df.describe(include=['O'])

* nameはデータセット全体で一意です（count = unique = 891）
  * わかる
* 65％が男性（top=male、freq= 577 / count= 891）
  * なるほど
* 客室の値はいくつかのトレーニングデータで二重になっています。 あるいは、複数の乗客が客室を共有しました。
  * わかる：Ticket
* Embarkedは3つの値をとる可能性があるが、 ほとんどの乗客がS港をから乗船しました。(top= S)
  * 644/669がSであってるのかな
* Ticket特徴量では、重複した値（ユニーク= 681 ）の割合（22％）が高い。
  * fm?
  
---

* Cabinは欠損が大きすぎるので使わない
* Ticketはユニーク値が少ない(681)ので使わない

# 相関を検証する

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

明らかに相関あり

In [ ]:
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

明らかに相関あり

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

SibSpとParchは特定の数値では無相関
一部の数値を用いた新しい特徴量を作るべき

# データ可視化

仮説検証のためにデータを可視化する

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

- 幼児の生存率が高い
- 最高年齢が生き残っている
- 多くの乗客が15~35周辺
- 15~25周辺の死亡率が高い

In [ ]:
g = sns.FacetGrid(train_df, col='Survived', row='Pclass')
g.map(plt.hist, 'Age', bins=20)
g.add_legend()

In [ ]:
g = sns.FacetGrid(train_df, col='Embarked')
g.map(sns.pointplot, 'Pclass', 'Survived', 'Sex')
g.add_legend()

- Embarked=Cはmaleとfemaleのバランスがが逆転する
- 全体的にfemaleのほうがmaleより生存している

In [ ]:
g = sns.FacetGrid(train_df, row='Embarked', col='Survived')
g.map(sns.barplot, 'Sex', 'Fare')
g.add_legend()

- Embarkedによって明らかに生存率が変化している
- FareとEmbarkedによって明らかに生存率が変化している
- Fareが上の方が生存率が高い

# データラングリング
## 使わない特徴量を削除する

In [ ]:
def dropDf(df):
    return df.drop(['Ticket', 'Cabin'], axis = 1)

train_df = dropDf(train_df)
test_df = dropDf(test_df)
combine = [train_df, test_df]

## 既存の特徴量から新しい特徴量を作成する
とりあえず名前の頭のやつを検証する

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)
    
pd.crosstab(train_df['Title'], train_df['Sex'])

In [ ]:
train_df[["Title", "Survived"]].groupby(['Title'], as_index=False).mean().sort_values(by='Survived', ascending=False)

明らかに生き残っているやつとそうでないやつがある👀

In [ ]:
train_df[["Title", "Age"]].groupby(['Title'], as_index=False).mean().sort_values(by='Age', ascending=False)

平均年齢は大きく差がある👀

使っていこう。
生存率を基準に名前を結合してみる

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Countess', 'Sir'], 'Good-Rare')
    dataset['Title'] = dataset['Title'].replace(['Capt','Don', 'Rev', 'Jonkheer'], 'Bad-Rare')
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Col','Dr', 'Major', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

```
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
```
このあたりはドメイン知識が無いと無理だな

とりあえずはこれらを数値に変換しておく。
せめて生存者割合逆順で。

TODO?: 非順序的情報のカテゴリ化

In [ ]:
title_mapping= {'Bad-Rare': 1, 'Mr': 2, 'Rare': 3, 'Master': 4, 'Miss': 5, 'Mrs': 6, 'Good-Rare': 7}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
train_df.head()

いらなくなった2行を削除。

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.head()

性別を数値に置き換え

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female': 1, 'male': 0}).astype(int)
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

数値連続的特徴量を補完するのに、ここでは3つの方法が考えられます。

1. 簡単な方法は、平均と標準偏差の間の乱数を生成することです。
2. 欠損値を推測するより正確な方法は、他の相関する特徴量を使用することです。今回のケースでは、年齢、性別、およびPclassの間の相関を記録する。 PclassとGenderの特徴量の組み合わせのセット全体でageの中央値を使用してAgeの値を推測します。Pclass = 1、Gender = 0、Pclass = 1、Gender = 1などの中間の年齢などなど。
3. 方法1と2を組み合わせる。中央値に基づいて年齢値を推測する代わりに、PclassとGenderの組み合わせのセットに基づいて、平均と標準偏差の間になる乱数を使用する。